# Aufgabe

In dem Aufgaben finden Sie 2 Dateien. Die Dateien enthalten englische Reviews von Filmen. Die eine Datei enthält negative die andere positive Reviews. 

Wir wollen einen Klassifikator bauen, der vorhersagt, ob ein Review positiv oder negativ ist. Machen Sie dazu folgendes:

1. Teilen Sie die Daten in Test und Trainingsdaten ein.
2. Extrahieren Sie alle Adjektive, Verben und Adverbien aus den Reviews
3. Nutzen Sie alle Adjektive, Verben und Adverbien, die mindesten 5 Mal vorkommen als Merkmale und Trainieren Sie einen Klassifikator, der logistische Regression nutzt.
5. Evaluieren Sie den Klassifikator

# Lösung

In [22]:
import codecs 

def readtext(dateiname):
    text = ''
    d = codecs.open(dateiname,'r','utf8')
    for zeile in d:
        text += str(zeile)
    d.close()

    return text

In [23]:
import glob

def read_data(directories):
    docs = []
    for directory in directories:
        for file in glob.glob(directory+"/*.txt"):
            text = readtext(file)
            docs.append((text,directory))
    return docs

data = read_data(['neg','pos'])

Teilen Sie die Daten in Test und Trainingsdaten ein.

In [24]:
import nltk
import re
from collections import Counter

stopwords = nltk.corpus.stopwords.words('english')

def features_from_text(text, POSlist):
    
    wordcounts = Counter()
    tlen = 0
    
    sentlist = nltk.sent_tokenize(text,language='english')
    for sent in sentlist:
        tokens = nltk.word_tokenize(sent,language='english')
        tokens = [word for (word,pos) in nltk.pos_tag(tokens) if pos in POSlist]
        tokens = [t for t in tokens if t.lower() not in stopwords]
        tokens = [t for t in tokens if re.search('^\w+$',t)]
        wordcounts.update(tokens)
        tlen +=len(tokens)
    return {w:wordcounts[w]/tlen for w in wordcounts}

In [25]:
import random

random.shuffle(data)
train_data_raw = data[1000:]
test_data_raw = data[:1000]

In [ ]:
poslist = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ','RB', 'RBR', 'RBS','JJ', 'JJR', 'JJS']
# poslist = ['RB', 'RBR', 'RBS']
# poslist = ['JJ', 'JJR', 'JJS']


test_data = [(features_from_text(text,poslist),cl) for text,cl in test_data_raw]
train_data = [(features_from_text(text,poslist),cl) for text,cl in train_data_raw]

In [ ]:
train_data[13]

In [ ]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

In [ ]:
docfreq.most_common(10)

In [ ]:
from sklearn import linear_model, datasets

allfeatures = [w for w in docfreq if docfreq[w] > 4]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

In [ ]:
train_vec[86]

In [ ]:
train_label[86]

In [ ]:
logreg = linear_model.LogisticRegression(C=1e9,verbose=True, max_iter=2000)
logreg.fit(train_vec,train_label)

In [ ]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

In [ ]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

In [ ]:
v = make_feat_vec(test_data[2][0],allfeatures) 
logreg.predict([v])

In [ ]:
test_data[2][1]